First preprocessing found here : https://towardsdatascience.com/deep-transfer-learning-for-natural-language-processing-text-classification-with-universal-1a2c69e5baa9

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Importation" data-toc-modified-id="Data-Importation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Importation</a></span></li><li><span><a href="#Neural-Network-for-text" data-toc-modified-id="Neural-Network-for-text-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Neural Network for text</a></span><ul class="toc-item"><li><span><a href="#Data-Preprocessing" data-toc-modified-id="Data-Preprocessing-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Data Preprocessing</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten, Embedding
from keras.layers import Input
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import layers, models, Model, optimizers
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
import tensorflow
import keras
import numpy as np
import argparse
import locale
import datetime
import os

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.


In [2]:
from numpy.random import seed
seed(1337)
import tensorflow as tf
tf.random.set_seed(42)

In [3]:
keras.backend.clear_session()

# Data Importation

In [4]:
data = pd.read_csv('data/data.csv')
target = pd.read_csv('data/target.csv')

# Neural Network for text

## Data Preprocessing

In [5]:
x_train = data[data['label'] == 'Train']['description']
x_val = data[data['label'] == 'Validation']['description']
x_test = data[data['label'] == 'Test']['description']

target.categ = pd.Categorical(target.categ)
target['categ_number'] = np.int32(target.categ.cat.codes)

y_train = np.asarray(target[target['label'] == 'Train'][['categ_number']])
y_val = np.asarray(target[target['label'] == 'Validation'][['categ_number']])
y_test = np.asarray(target[target['label'] == 'Test'][['categ_number']])

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)

X_train = tokenizer.texts_to_sequences(x_train)
X_val = tokenizer.texts_to_sequences(x_val)
X_test = tokenizer.texts_to_sequences(x_test)

maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

vocab_size = len(tokenizer.word_index) + 1

In [7]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, 
                    output_dim=50, 
                    input_length=maxlen))
#model.add(layers.GlobalMaxPool1D())
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           232200    
_________________________________________________________________
flatten_1 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               500100    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_5 (Dense)              (None, 7)                

In [8]:
class PrintLogs(tf.keras.callbacks.Callback):
    def __init__(self, epochs):
        self.epochs = epochs

    def set_params(self, params):
        params['epochs'] = 0

    def on_epoch_begin(self, epoch, logs=None):
        print('Epoch %d/%d' % (epoch + 1, self.epochs), end='')

In [9]:
epochs = 10

history = model.fit(X_train, y_train,
                    epochs=epochs,
                    validation_data=(X_val, y_val),
                    callbacks=[PrintLogs(epochs)],
                    batch_size=32,
                    verbose=1)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 630 samples, validate on 210 samples
Epoch 1/0
630/630 [==============================] - 0s 757us/step - loss: 1.9182 - accuracy: 0.1635 - val_loss: 1.8787 - val_accuracy: 0.1857
Epoch 2/0
630/630 [==============================] - 0s 88us/step - loss: 1.8000 - accuracy: 0.2063 - val_loss: 1.7994 - val_accuracy: 0.2143
Epoch 3/0
630/630 [==============================] - 0s 110us/step - loss: 1.6662 - accuracy: 0.2873 - val_loss: 1.7114 - val_accuracy: 0.3381
Epoch 4/0
630/630 [==============================] - 0s 99us/step - loss: 1.4600 - accuracy: 0.4667 - val_loss: 1.5504 - val_accuracy: 0.3810
Epoch 5/0
630/630 [==============================] - 0s 85us/step - loss: 1.1798 - accuracy: 0.6016 - val_loss: 1.3906 - val_accuracy: 0.4905
Epoch 6/0
630/630 [==============================] - 0s 74us/step - loss: 0.8481 - accuracy: 0.6714 - val_loss: 1.1870 - val_accuracy: 0.5048
Epoch 7/0
630/630 [==============================] - 0s 85us/step - loss: 0.6003 - accuracy: 0.7206 

In [10]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

630/630 [==============================] - 0s 25us/step
Training Accuracy: 0.8841
210/210 [==============================] - 0s 74us/step
Testing Accuracy:  0.6952
